In [1]:
import os

In [2]:
%pwd

'/workspaces/Text_summarization_Demo/research'

In [3]:
os.chdir('../')

In [4]:
%pwd

'/workspaces/Text_summarization_Demo'

# Entity

Entity(varlıklar) projenin değişkenleri ve değişkenlerin tipleri merkezi yapıda tutulmasını sağlar ve saklar

In [5]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen = True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path


In [6]:
from src.textSummarizer.constants import * # * demek bütün değişkenleri içe aktarmak demektir.
from src.textSummarizer.utils.common import read_yaml, create_directories

# Configration Manager
Entity'de oluşturmuş olduğumuz değişkenleri tiplerine uygun bir şekilde değerlerini bu aşama içerisinde ayarlıyoruz. Bu değişkenlerin değerlerini ayarlarken ilk başta oluşturmuş olduğumuz yaml dosyalarından değerleri okuyoruz.

In [7]:
class ConfigurationManager:
    def __init__(
            self,
            config_filepath = CONFIG_FILE_PATH,
            params_filepath = PARAMS_FILE_PATH
    ):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])

    def get_data_ingestion_config(self)->DataIngestionConfig:
        config = self.config.data_ingestion
        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir
        )
        return data_ingestion_config

# Components
Artık değerleri belli olan değişkenlerimi kullanarak yapmak istediğim asıl işe başlayabilirim.

In [8]:
import os
import urllib.request as request
import zipfile
from src.textSummarizer import logger
from src.textSummarizer.utils.common import get_size

In [9]:
class DataIngestion:
    def __init__(self, config:DataIngestionConfig):
        self.config = config


    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url = self.config.source_URL,
                filename=self.config.local_data_file
            )
            logger.info(f"{filename} dosyası indirildi. {headers} bilgilerini içeriyor.")
        else:
            logger.info(f'Böyle dosya zaten mevcut. Dosyanın boyutu: {get_size(Path(self.config.local_data_file))}')

    def exract_zip_file(self):
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, "r") as zip_ref:
            zip_ref.extractall(unzip_path)

# Pipeline

components tanımlanmış olan işlemleri aşamalı olarak çalıştırmaya yarar.

In [10]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.exract_zip_file()
except Exception as e:
    raise e

[2024-11-18 20:20:13,365: INFO: common: yaml file: config/config.yaml loaded successfully]
[2024-11-18 20:20:13,371: INFO: common: yaml file: params.yaml loaded successfully]
[2024-11-18 20:20:13,371: INFO: common: created directory at: artifacts]
[2024-11-18 20:20:13,373: INFO: common: created directory at: artifacts/data_ingestion]
[2024-11-18 20:20:13,933: INFO: 1260393074: artifacts/data_ingestion/data.zip dosyası indirildi. Connection: close
Content-Length: 7903594
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "dbc016a060da18070593b83afff580c9b300f0b6ea4147a7988433e04df246ca"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: 2A4D:2BFC2D:72602A:763739:673B9C67
Accept-Ranges: bytes
Date: Mon, 18 Nov 2024 20:20:13 GMT
Via: 1.1 varnish
X-Served-By: cache-ams2100100-AMS
X-Cache: HIT
X-Cache-